In [1]:
#!conda activate implicit_39
#!pip install pandas numpy matplotlib scipy implicit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
# !conda install -c conda-forge implicit --all --yes
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('../_ADDS/webinar_2/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    recs = np.random.choice(a=np.array(items_weights['item_id']),
                            p=np.array(items_weights['weight']),
                            size=n, 
                            replace=False)
    return recs.tolist()

In [5]:
%%time
# your_code

# группировка по товарам и расчет логарифма
items_weights = data_train.pivot_table(index='item_id', values='sales_value', aggfunc='sum')
items_weights['log'] = np.log2(items_weights['sales_value'])

# замена минус бесконечности (результат логарифма 0 по 2) на минимальное значение по колонке
minimal_val = items_weights.loc[items_weights['log'] != -np.Inf, 'log'].min()
minimal_val -= 1
items_weights.loc[items_weights['log'] == -np.Inf, 'log'] = minimal_val

# scaling
items_weights['weight'] = items_weights['log'] + abs(minimal_val)
agg_val = items_weights['weight'].sum()
items_weights['weight'] = items_weights['weight'] / agg_val

# очистка от лишних колонок и форматирование таблицы
items_weights = items_weights.reset_index()[['item_id', 'weight']]


# формирование рекомендаций
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
items = data_train.item_id.unique()
result['w_random_recommendation'] = result['user_id']\
                                  .apply(lambda x: 
                                         weighted_random_recommendation(items_weights, n=5))
result.head(2)

/home/arthur/.conda/envs/implicit_39/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


CPU times: user 1.7 s, sys: 19.6 ms, total: 1.72 s
Wall time: 1.72 s


,user_id,actual,w_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1984332, 969263, 938381, 1037636, 15483442]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[893110, 838811, 1032185, 6464714, 1026890]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [6]:
result = pd.read_csv('../_ADDS/webinar_2/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [7]:
# your_code
def eval_actual(item):
    garbage = ['[', ']', '\n']
    for g in garbage:
        item = item.replace(g, '')
    item = list(map(int, item.split()))
    return item

def precision_at_k(x, k=5):
    recommended_list = x[0]
    bought_list = x[1]
    
    bought_list = eval_actual(bought_list)
    bought_list = np.array(bought_list)
    
    recommended_list = eval(recommended_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[:k]
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    return precision

k = 5
model_names = result.columns.tolist()[2:]
for model in model_names:
    result[f'{model}_precision@{k}'] = result[[model, 'actual']].apply(precision_at_k, axis=1)

model_mask = ['@' in col for col in result.columns]
cols = result.columns.tolist()
model_cols = [cols[i] for i in range(len(cols)) if model_mask[i]]

result[model_cols].mean()

random_recommendation_precision@5     0.000588
popular_recommendation_precision@5    0.155240
itemitem_precision@5                  0.033595
cosine_precision@5                    0.035260
tfidf_precision@5                     0.036141
own_purchases_precision@5             0.179987
dtype: float64